# **PRP Assignmnet-4: Image Denoising by Non-Local Means**

In [1]:
#### Different Librarires ####
from numba import jit
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

Note: Run all the cells one by one.

# **Extracting Files**

Here we access the files from GitHub Repository.

In [2]:
## Cloning GitHub Repository ##
!git clone --quiet https://github.com/sanket2812/Image_Denoising_Non-Local-Means



# **Gaussian Weight Calculating Function**

This calculates the Gaussian weights for a particular size of window. This helps in increaseing the contribution of the nearby pixels rather then the pixels which are far from the given pixel. 

In [3]:
def gauss_weights_calculate(win_size=(7,7),sigma=20):    
    err = 1e-16                                                                 #Error term (User can change this according to his/her convenience)
    v1 , v2 = [(i-1)/2.0 for i in win_size]
    y,x = np.ogrid[-v1:v1+1 , -v2:v2+1]
    g_weights = np.exp(-(x*x + y*y)/(2.0 * sigma**2))                           #Calculating Weigths
    g_weights[ g_weights < err ] = 0                                            #If weights are less then a certain amount(error) make it zero
    sum = g_weights.sum()
    if sum != 0:
      g_weights = g_weights/sum
      
    return g_weights

# **Window Making Functions**

This function will create a square window and the center of the window will be the given pixel.

Both search window and neighbourdhood window making functions are in this cell.

In [4]:
@jit(nopython=True, cache=True)
def make_window(pic,y,x,win_size):
    window = np.zeros((win_size,win_size))                                      #Finding picture dimension 
    h, w = pic.shape                                                            #Initializing Window
    
    middle = win_size//2                                                        #Middle of Window

    xmax = min(w, x + middle+1)                                                 #Four 
    xmin = max(0, x - middle)                                                   #Coordinates 
    ymax = min(h, y + middle+1)                                                 #of the 
    ymin = max(0, y - middle)                                                   #window
    
    pxmax = middle + (xmax-x)
    pxmin = middle - (x-xmin)    
    pymax = middle + (ymax-y)
    pymin = middle - (y-ymin)

    
    window[pymin:pymax , pxmin:pxmax] = pic[ymin:ymax , xmin:xmax]
    
    return window

@jit(nopython=True, cache=True)
def search_window(neighbours,y,x,se_win_size,win_size):
    window = np.zeros((se_win_size,se_win_size,win_size,win_size))              #Finding picture dimension   
    h, w = neighbours.shape[0],neighbours.shape[1]                              #Initializing Window  
        
    middle = se_win_size//2                                                     #Middle of Window

    xmax = min(w, x + middle+1)                                                 #Four 
    xmin = max(0, x - middle)                                                   #Coordinates 
    ymax = min(h, y + middle+1)                                                 #of the   
    ymin = max(0, y - middle)                                                   #window
    
    pxmax = middle + (xmax-x)
    pxmin = middle - (x-xmin)    
    pymax = middle + (ymax-y)
    pymin = middle - (y-ymin)
    
    window[pymin:pymax,pxmin:pxmax,:,:] = neighbours[ymin:ymax,xmin:xmax,:,:]
    
    return window

# **Finding Neighbours**

This function finds the neighbours of every pixel in the given image. The neighbours are in a square window, having the selected pixel at the center of the window.


In [5]:
@jit(nopython=True, cache=True)
def find_neighbours(pic,win_size):
    h,w = pic.shape
    neighbour = np.zeros((h,w,win_size,win_size))
    for y in range(h):
        for x in range(w):
            neighbour[y,x] = make_window(pic,y,x,win_size)
    
    return neighbour

# **Similarity Weight Calculating Function**

This function find the weight of each pixel which are in the square neighbourhood window of the selected pixel.Weights are calculated depending upon the similarity of the intensity level of the selected pixels to the intensity level of the neighbourhood pixels.

In [6]:
@jit(nopython=True, cache=True)
def weight_similar(current_window, window, gauss_weights,h=12):                 # The filtering parameter h can be changed depending 
                                                                                # upon the requirement
    norm = ((np.square(current_window - window) * gauss_weights).sum(axis = 3)).sum(axis=2)
    norm = norm/h**2
    similar = np.exp(-norm)
    Z = np.sum(similar)
    weight = similar/Z
    return weight

# **Filtering Algorithms**

## **Gaussian Filtering**

The gaussian filtering is done here.

In [7]:
def gaussian_filtering(pic,sigma=1):                                            # The Gaussian Filtering Parameter sigma can be changed here
                                                                                # according to the requirement  
    return gaussian_filter(pic, sigma)
  

## **Non-Local Means**
This is the main function through which different functions will be called and finally the image will be filtered by non-local means algorithm.

In [8]:
def NLM(pic):
    win_size=7                                                                  #Neighbourhood Window Size , Can change it from here
    se_win_size=21                                                              #Search Window Size, Can change it from here
    h,w = pic.shape
    pic_evalute = np.copy(pic)
    neighbour = find_neighbours(pic,win_size)                                   #Finding the neighboours of all the pixel
    gauss_weights = gauss_weights_calculate((win_size,win_size))                #Finding Gaussian weights
    for y in range(h):
        for x in range(w):
            presnt_wind = neighbour[y,x,:,:]
            se_neighbours = search_window(neighbour,y,x,se_win_size,win_size)   #Making Search Window
            pixel_neighb = make_window(pic_evalute,y,x,se_win_size)                       
            weight = weight_similar(se_neighbours,presnt_wind,gauss_weights)    #Finding the weights according to the similarity of intensity level 
            pic_evalute[y,x] = int(np.sum(weight * pixel_neighb))               #Final value of pixel
    
    return pic_evalute

# **MSE and PSNR Calculating Function**

This function calculates the MSE and PSNR between two given images, for our case one is noisy image and the other is ground truth.

In [9]:
def mse_psnr(pic, gnd_tru):
    max_pix = 255.0
    mse = np.mean((pic.astype(float) - gnd_tru.astype(float)) ** 2) 
    psnr = 20 * np.log10(max_pix / np.sqrt(mse)) 
    
    return mse, psnr

# **Main Cell**

This is the main cell where we read the ground truth and noisy image and then call the NLM and Gaussian Filtering functions for removing noise and then compare the results by calculating MSE and PSNR.

Note: This will take a few minutes to run and give the output.

In [ ]:
## Change the path here if you want to perform it with a different image ##
g_truth = cv2.imread("/content/Image_Denoising_Non-Local-Means/Ground_Truth/Image1.png",cv2.IMREAD_GRAYSCALE)             ## Add ground truth here
noise_pic = cv2.imread("/content/Image_Denoising_Non-Local-Means/Gaussian_Noise_Images/Image1.png",cv2.IMREAD_GRAYSCALE)  ## Add image having noise here
noise_score = mse_psnr(noise_pic,g_truth)
nlm_filter = NLM(noise_pic)
nlm_mse_psnr = mse_psnr(nlm_filter, g_truth)
gauss_filter = gaussian_filtering(noise_pic)
gauss_mse_psnr = mse_psnr(gauss_filter, g_truth)

## Parameters Used for Filtering the Image ##
Gauss_param_sigma = 1
h = 12

## Making the Plot ## 
fig = plt.figure(figsize=(12,12))
plt.suptitle('NLM Filtering Parameter h = {}, Gaussian Filtering Sigma = {}'.format(h,Gauss_param_sigma),fontsize=16,y = 0.94)

## Figure 1: Ground Truth ##
ax1 = fig.add_subplot(2,2,1, title = "Ground Truth")
plt.xticks([])
plt.yticks([])
ax1.imshow(g_truth, cmap = 'gray')
## Figure 2: NLM Filtered Image ##
ax2 = fig.add_subplot(2,2,2, title = "NLM Filtering \nMSE: {:.2f}, PSNR: {:.2f}".format(nlm_mse_psnr[0],nlm_mse_psnr[1])) 
plt.xticks([])
plt.yticks([])
ax2.imshow(nlm_filter, cmap = 'gray')
## Figure 3: Image with Noise ##
ax3 = fig.add_subplot(2,2,3, title = "Image with Noise \nMSE: {:.2f}, PSNR: {:.2f}".format(noise_score[0],noise_score[1])) 
plt.xticks([])
plt.yticks([])
ax3.imshow(noise_pic, cmap = 'gray')

## Figure 4: Gauss Filtered Image ##
ax4 = fig.add_subplot(2,2,4, title = "Gaussian Filtering \nMSE: {:.2f}, PSNR: {:.2f}".format(gauss_mse_psnr[0],gauss_mse_psnr[1]))
ax4.imshow(gauss_filter, cmap = 'gray')
plt.xticks([])
plt.yticks([])
plt.savefig("Result.png")